In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import missingno as msno
from collections import Counter
from itertools import chain, combinations
import sklearn as sk
from sklearn.metrics import accuracy_score, f1_score

# Performance comparison

In this notebook we compare the performance of multiple models on different subsets of our data.
#### The models are:
+ Logistic Regression
+ SVM
+ KNN
+ Neural Network
+ XGBoost

#### The datasets:
+ Mean/Mode imputed
+ KNN imputed
+ MICE imputed
+ no imputations

#### The targets:
+ has_dep_diag
+ a binary comination of all the target variables

#### In combinations of:
+ trained on balanced, tested on balanced
+ trained on balanced, tested on imbalanced
+ trained on imbalanced, tested on balanced
+ trained on imbalanced, tested on imbalanced

#### We employ  range of visualisation methods:
+ ROC curves
+ bar plots
+ learning curves

In [ ]:
data = pd.read_csv("imputed_dataset_5.csv", engine='python').drop(['Unnamed: 0'], axis = 1)
dep_data = data.copy()

dep_num = np.array([12, 11, 10, 4])
no_dep_num = np.setdiff1d(range(13), dep_num)

dep_data[['prim_diag', 'secd_diag']] = dep_data[['prim_diag', 'secd_diag']].replace(list(no_dep_num), 0)
dep_data[['prim_diag', 'secd_diag']] = dep_data[['prim_diag', 'secd_diag']].replace(list(dep_num), 1)

In [ ]:
full = data.dropna(axis = 0, how = 'any')
dep = [x for x in data.columns if 'dep' in x or 'diag' in x or 'panic' in x]
dep_data['dep'] = dep_data['secd_diag'] + dep_data['prim_diag'] + 0 * dep_data['has_dep_diag']
dep_data['dep'] = dep_data['dep'].replace(range(2, 4), 1)
full = dep_data.dropna(axis = 0, how = 'any')
dep = [x for x in data.columns if 'dep' in x or 'diag' in x or 'panic' in x]

In [ ]:
has_dep = full.query('dep == 1')
no_dep = full.query('dep == 0')
size = int(np.round(0.8 * min(len(has_dep), len(no_dep))))
#sample = pd.concat([has_dep.sample(size), no_dep.sample(size)])
sample = full.sample(8000)

sample = sample.sort_index()
sample = sample.reset_index(drop = True)


X = sample.drop(dep, axis = 1).drop('dep', axis = 1)
y = sample['dep']

#sample = full.sample(8000)
#smote = SMOTE(random_state = 0)
#X, y = smote.fit_resample(sample.drop(dep, axis = 1).drop('dep', axis = 1), sample['dep'])